# Question 1

In [ ]:
#blast search generated seqdump_1.txt

# Question 2

In [ ]:
%%bash
transeq ~/Downloads/seqdump.txt > protein.fa
mafft protein > protein_align.fa

# Question 3

In [10]:
from fasta_iterator_class import FASTAReader

ModuleNotFoundError: No module named 'fasta_iterator_class.py'; 'fasta_iterator_class' is not a package

In [12]:
import sys

class FASTAReader(object):

    def __init__(self, file):
        self.last_id = None
        self.file = file
        self.eof = False

    def __iter__(self):
        return self

    def __next__(self):
        if self.eof:
            raise StopIteration
        # check if this is the first sequence from the file
        if self.last_id is None:
            # First line
            line = self.file.readline()
            # Verify that this is a FASTA file
            assert line.startswith(">"), "Not a FASTA file"
            # Get the sequence ID
            seq_id = line[1:].rstrip("\r\n")
        else:
            # Get ID from previous round
            seq_id = self.last_id

        sequence = []
        while True:
            line = self.file.readline()
            # Check if we've reached the end of the file
            if line == "":
                self.eof = True
                break
            # Check if we've reached the next sequence
            elif not line.startswith(">"):
                sequence.append(line.strip())
            # We've reached the next sequence ID
            else:
                self.last_id = line[1:].rstrip("\r\n")
                break
        
        sequence = "".join(sequence)
        return seq_id, sequence

if __name__ == "__main__":
    reader = FASTAReader(sys.stdin)

    for seq_id, sequence in reader:
        print(seq_id, sequence)


AssertionError: Not a FASTA file

In [62]:
def FASTAReader(file):
    line = file.readline()
    assert line.startswith('>'), "Not a FASTA file"
    seq_id = line[1:].rstrip('\r\n')
    sequence = []
    line = file.readline()
    sequences = []
    while line:
        if line.startswith('>'):
            sequences.append((seq_id, ''.join(sequence)))
            seq_id = line[1:].rstrip('\r\n')
            sequence = []
        else:
            sequence.append(line.strip())
        line = file.readline()
    sequences.append((seq_id, ''.join(sequence)))
    return sequences

In [63]:
aa_seq = FASTAReader(open("protein_align.fa"))
org_nuc_seq = FASTAReader(open("seqdump_1.txt"))

In [64]:
seq_dict = {}

#make dictionary with original DNA sequence(key) and amino acid sequence(value), then later append dictionary with third query sequence
for seq_id,sequence in org_nuc_seq:
    seq_id = seq_id.split()
    seq_dict[seq_id[0]] = [sequence]
    
for seq_id,sequence in aa_seq:
    seq_id = seq_id.split()
    seq_dict[seq_id[0][:-2]].append(sequence)

seq_dict["NC_001563.2"] #print first sequence
    

#need to remove ID from line in sequence
#for sequence id in fasta reader (object) 
#put into a dictionary 
#key is id with list of orignal nuc then amino acid sequence



['AGTAGTTCGCCTGTGTGAGCTGACAAACTTAGTAGTGTTTGTGAGGATTAACAACAATTAACACAGTGCGAGCTGTTTCTTGGCACGAAGATCTCGATGTCTAAGAAACCAGGAGGGCCCGGTAAAAACCGGGCTGTCAATATGCTAAAACGCGGTATGCCCCGCGGATTGTCCTTGATAGGACTAAAGAGGGCTATGCTGAGTCTGATTGACGGGAAGGGCCCAATACGTTTCGTGTTGGCTCTTTTGGCGTTTTTCAGATTCACTGCAATCGCTCCGACTCGTGCGGTGCTGGACAGATGGAGAGGCGTCAACAAACAAACAGCAATGAAGCATCTCTTGAGTTTCAAGAAAGAACTAGGAACTCTGACCAGTGCCATCAACCGCCGGAGCACAAAACAAAAGAAAAGAGGAGGCACAGCGGGCTTTACTATCTTGCTTGGGCTGATCGCCTGTGCTGGAGCTGTGACCCTCTCGAACTTCCAGGGCAAAGTGATGATGACAGTCAATGCAACCGATGTCACTGACGTGATTACCATTCCAACAGCTGCTGGGAAAAACCTGTGCATCGTAAGGGCTATGGACGTAGGATACCTTTGTGAGGATACTATCACTTATGAATGTCCGGTCCTAGCTGCTGGAAATGACCCTGAAGACATTGACTGCTGGTGCACGAAATCATCTGTTTACGTGCGCTATGGAAGATGCACAAAAACTCGGCATTCCCGTCGAAGCAGAAGGTCTCTGACAGTCCAGACACATGGAGAAAGTACACTGGCCAACAAGAAAGGAGCTTGGTTGGACAGCACAAAAGCCACGAGATATCTGGTGAAGACAGAATCATGGATACTGAGAAACCCGGGCTACGCCCTCGTTGCAGCTGTCATTGGATGGATGCTAGGAAGCAACACAATGCAACGCGTCGTGTTTGCCATTCTATTGCTCCTGGTGGCACCAGCATACAGCTTCAACTGTTTAGGAATGAGTAACAGAGACTT

In [65]:
#make list with new nucleotide sequence in it to add to dictionary from eariler 
#new_nuc_list: ""
for key, value in seq_dict.items():
    new_nuc_seq = '' 
    org_nuc_seq = value[0]
    aa_seq = value[1]
    for i,j in enumerate(aa_seq):
        if j == '-':
            new_nuc_seq +='---'#if there is a gap in the aa then add three gaps in the new nuc sequence
        else:
            new_nuc_seq += org_nuc_seq[i:(i+3)] #move to next codon in new sequence and add nucleotides from original
    
            
            
    seq_dict[key].append(new_nuc_seq)

seq_dict["NC_001563.2"] #print first sequence, printing the whole dictionary made jupyter hate me, trying to keep her happy
    


['AGTAGTTCGCCTGTGTGAGCTGACAAACTTAGTAGTGTTTGTGAGGATTAACAACAATTAACACAGTGCGAGCTGTTTCTTGGCACGAAGATCTCGATGTCTAAGAAACCAGGAGGGCCCGGTAAAAACCGGGCTGTCAATATGCTAAAACGCGGTATGCCCCGCGGATTGTCCTTGATAGGACTAAAGAGGGCTATGCTGAGTCTGATTGACGGGAAGGGCCCAATACGTTTCGTGTTGGCTCTTTTGGCGTTTTTCAGATTCACTGCAATCGCTCCGACTCGTGCGGTGCTGGACAGATGGAGAGGCGTCAACAAACAAACAGCAATGAAGCATCTCTTGAGTTTCAAGAAAGAACTAGGAACTCTGACCAGTGCCATCAACCGCCGGAGCACAAAACAAAAGAAAAGAGGAGGCACAGCGGGCTTTACTATCTTGCTTGGGCTGATCGCCTGTGCTGGAGCTGTGACCCTCTCGAACTTCCAGGGCAAAGTGATGATGACAGTCAATGCAACCGATGTCACTGACGTGATTACCATTCCAACAGCTGCTGGGAAAAACCTGTGCATCGTAAGGGCTATGGACGTAGGATACCTTTGTGAGGATACTATCACTTATGAATGTCCGGTCCTAGCTGCTGGAAATGACCCTGAAGACATTGACTGCTGGTGCACGAAATCATCTGTTTACGTGCGCTATGGAAGATGCACAAAAACTCGGCATTCCCGTCGAAGCAGAAGGTCTCTGACAGTCCAGACACATGGAGAAAGTACACTGGCCAACAAGAAAGGAGCTTGGTTGGACAGCACAAAAGCCACGAGATATCTGGTGAAGACAGAATCATGGATACTGAGAAACCCGGGCTACGCCCTCGTTGCAGCTGTCATTGGATGGATGCTAGGAAGCAACACAATGCAACGCGTCGTGTTTGCCATTCTATTGCTCCTGGTGGCACCAGCATACAGCTTCAACTGTTTAGGAATGAGTAACAGAGACTT

In [60]:
print(new_nuc_seq)

---------------------------------------------------------------------------------------------------------------TTTTTGTGTGTGTGA------------------------------------------------------------------AGTGTGTGCGCGCGAGAGAGCGCT------------------CTTTTA------GCACACACG------------------------TCGCGAGATATGTGTGTCTCTCTATAAAAGAGAGAAAAAAACACC------------------------GCCCCCCCG---------TAAAAAAAAAAAAACACCCCGCGGGGGGGCGCTCTGTGT---TCACAA---ATATATATGTGCGCTCTATAAAAAAAA------------------------ATGTGCGCCCCCCCCCCGCGC---------------------------------TTGTGAGATATATAGAGGGGAGACACTCTGTGAGAAAAGAGAGAGAGGGGGGGCGCTCTATATATGTGC---CTGTGAGAGAGTGTCTCTCTGTGAGATATTTTGTGAGACACGCGGGGAGAAAAAAAAAAGAGGGGCGCCCCCCCACAAAATATATACACGCGTGTTTTTTTCTCGCGTGTGTGTGTTTTGTGGGGCGCTCTCTCTCTTTTTTTTTTGTGGGGCGCGCGTGTTTTTTTTTTTTTCTCACAGAGAGATATTTTCTCACACACTCTGTGCGCACAAAATATCTCGCGCGCTCTCTCCCCGCGAGACACTCTCTCGCGTGTGTGCGCGCGGGGTGTGTGCGCTCTGTGGGGAGACACACAGAGAGATATGTGGGGAGAGAGAGAGAGGGGCGCGCGTGTCTCACAAAACACACAAAAAAAC---CAAAAAAACACACAGAGCGCGCGAGATATGTGAGAAAAGAGCGCAC

# Question 4

In [ ]:
#want to go thruhg each sequence or three sets of places in sequence 
dN if 
if queery has gap skiiiip()
for query at end have dn ds for each dn ds
score of 1 if change in amino acid 
score of 0 is no change in coded amino acid

if three gaps contine
if same continue
if aa equal the ds 
other wise = dn
how to loop  through 
6008 codons, for loop 0, 6008, for loop dictionary range, divide by 3 

In [66]:
dN_list = [] 
dS_list = []
codon_list = []
org_codon = seq_dict.get('NC_001563.2')[0] #making reference sequence of original sequence that has no gaps from first list in dictionary
new_codon = seq_dict.get('NC_001563.2')[2] #making reference of new nucleotide sequence with gaps from third list in dictionary
#orgcodon2 = org_codon[i:(i+3)] #codon = three nucleotides 
#newcodon2 = new_codon[i:(i+3)] #codon = three nucleotides
codon_count = 0

for i in range(0, len(org_codon), 3): #iterate over codons in nucleotife sequence without gaps 
     
    #make counts forcodons, dS, and dN
    dN_count = 0
    dS_count = 0
    codon_count += 1

    if new_codon[i:(i+3)] == '---': #ignore if gaps in new sequence and do not need to compare to reference
        continue
    for key, value in seq_dict.items():
        if key == 'NC_001563.2':
            continue
        else:
            org_nuc = value[0] #only using first list in dictionary 
            #aa = value[1] #second list in dicitonary 
            new_nuc = value[2] #third list in dictionary 
           # org_nuc2 = org_nuc[i:(i+3)] #defining codon length
           # new_nuc2 = new_nuc[i:(i+3)] #defining codon length 
            
            if org_nuc[i:(i+3)] == org_codon[i:(i+3)]:
                continue
            elif org_nuc[i:(i+3)] == '---':
                continue
            elif org_nuc[i:(i+3)] == org_codon[i:(i+3)]:
                continue
            elif new_nuc[i:(i+3)] == new_codon[i:(i+3)]:
                dS_count += 1
            else:
                dN_count += 1
    codon_list.append(codon_count)
    dS_list.append(dS_count)
    dN_list.append(dN_count)


In [67]:
print(dS_list)

[2, 2, 6, 294, 290, 9, 1, 288, 280, 13, 282, 13, 291, 1, 0, 5, 1, 1, 287, 284, 286, 285, 5, 6, 289, 287, 1, 6, 298, 296, 16, 5, 6, 2, 6, 282, 22, 4, 10, 29, 40, 27, 5, 482, 3, 4, 286, 2, 8, 13, 4, 7, 290, 294, 293, 296, 293, 309, 292, 306, 285, 6, 11, 289, 25, 23, 283, 286, 303, 334, 288, 6, 2, 28, 6, 3, 7, 29, 7, 10, 10, 286, 6, 323, 334, 297, 284, 1, 277, 318, 0, 16, 297, 290, 4, 5, 1, 8, 22, 8, 2, 10, 6, 294, 294, 12, 4, 3, 84, 291, 4, 2, 0, 0, 1, 1, 4, 282, 283, 3, 292, 30, 39, 51, 25, 21, 25, 30, 5, 294, 331, 334, 328, 473, 474, 354, 1, 11, 22, 4, 287, 12, 38, 285, 43, 6, 1, 10, 20, 306, 300, 1, 5, 12, 20, 289, 286, 4, 0, 12, 13, 9, 285, 2, 13, 0, 287, 22, 279, 288, 295, 8, 0, 1, 1, 0, 3, 0, 0, 7, 5, 299, 15, 288, 290, 8, 307, 4, 5, 2, 13, 6, 289, 277, 11, 5, 288, 10, 289, 2, 4, 0, 3, 6, 66, 30, 26, 279, 290, 17, 10, 291, 16, 10, 22, 31, 4, 6, 10, 5, 295, 20, 20, 0, 2, 6, 2, 16, 2, 2, 1, 23, 86, 5, 288, 9, 285, 11, 190, 185, 50, 317, 297, 12, 16, 11, 287, 5, 290, 307, 18, 15, 2, 6

In [49]:
print(value[1])

-------------------------------------SSSPV----------------------ADKLSSVC------ED--QLT--------QCELFLSTKISMSKK--------PGG---PGKNRAVNMLKRG-MP-RGLSLIGLK--------RAMLSLI-----------DGKGPIRFVLALLAFFRFTAIAP-TRAVLDRWRGVNKQTAMKHLLSFKKELGTLTSAINRRSTKQKKRGGTAGFTILLGLIACAGAVTLSNFQGKVMMTVNATDVADVITIPTAAGKNLCIVRAMDVGYLCEDTITYECPVLA-AGNDPEDIDCWCTKSSVYVRYGRCTKTRHSRRSRRSLTVQ-THGESTLANKKGAWLDSTKATRYLVKTESWILRNPGYALVAAVIGWMLGSNTMQRVVFAILLLLVAPAYSFNCLGMSNRDFLEGVSGATWVDLVLEGDSCVTIMSKDKPTIDVKMMNMEAANLADVRSYCY-----LASVSDLSTRAACPTMGEAHN---EKRADPAFVCKQGVVDRGWGNGCGLFGKGSIDTCAKFAC-TTKATGWIIQKENIKYEVAIFVHGPTTVES----HGNYSTQTGATQAGRFSITPSAPSYTLKLGEYGEVTVDCEPR--------SGIDTSAYYVMSVGAKSFLVHREWFMDLNLPW-SSAGSTTWR----NRETLMEFEEPHATK---------Q--SVVALGSQEGAL-------HQALAGAIPVEFSSNT-----VKLTSGHLKCRVKMEKLQLKGTT-YGV-------CSKAFKFAGTPADTGHGTVVLELQYT------------GTDGP-CKVPISSVASLNDLTPV-G-RLVTV-------NPFVSVATANSKVLIELEPPFGDSYIV-----VGRGEQQINHHWHKSGSSIGK----AFT-----------------------------TTLRGAQ-RLAALGDTAWD---FG---------------------

In [68]:
print(dS_list)

[2, 2, 6, 294, 290, 9, 1, 288, 280, 13, 282, 13, 291, 1, 0, 5, 1, 1, 287, 284, 286, 285, 5, 6, 289, 287, 1, 6, 298, 296, 16, 5, 6, 2, 6, 282, 22, 4, 10, 29, 40, 27, 5, 482, 3, 4, 286, 2, 8, 13, 4, 7, 290, 294, 293, 296, 293, 309, 292, 306, 285, 6, 11, 289, 25, 23, 283, 286, 303, 334, 288, 6, 2, 28, 6, 3, 7, 29, 7, 10, 10, 286, 6, 323, 334, 297, 284, 1, 277, 318, 0, 16, 297, 290, 4, 5, 1, 8, 22, 8, 2, 10, 6, 294, 294, 12, 4, 3, 84, 291, 4, 2, 0, 0, 1, 1, 4, 282, 283, 3, 292, 30, 39, 51, 25, 21, 25, 30, 5, 294, 331, 334, 328, 473, 474, 354, 1, 11, 22, 4, 287, 12, 38, 285, 43, 6, 1, 10, 20, 306, 300, 1, 5, 12, 20, 289, 286, 4, 0, 12, 13, 9, 285, 2, 13, 0, 287, 22, 279, 288, 295, 8, 0, 1, 1, 0, 3, 0, 0, 7, 5, 299, 15, 288, 290, 8, 307, 4, 5, 2, 13, 6, 289, 277, 11, 5, 288, 10, 289, 2, 4, 0, 3, 6, 66, 30, 26, 279, 290, 17, 10, 291, 16, 10, 22, 31, 4, 6, 10, 5, 295, 20, 20, 0, 2, 6, 2, 16, 2, 2, 1, 23, 86, 5, 288, 9, 285, 11, 190, 185, 50, 317, 297, 12, 16, 11, 287, 5, 290, 307, 18, 15, 2, 6

In [ ]:
print(dN_list)